![banner_etl](https://github.com/cistelsa/Commerce_Data_Analysis_and_Recommendations/blob/main/5_Sources/Images/banner_automatizacion.gif?raw=true)

**<mark style="background:#2bfe9c">Script de automatización</mark>**

# <h2 align=center>**`Agregar 'hotel_id' de Hotel Beds a (Google Maps, Yelp)`**</h2>

Importar librerías

In [ ]:
pip install geopandas

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 5, Finished, Available)

Cargar el dataset de Google Maps

In [44]:
hotels_gm = pd.read_csv("/lakehouse/default/Files/data/beta/Google/hotels_gm.csv")
hotels_gm.drop_duplicates(subset=['gmap_id'], inplace=True, ignore_index=True)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 46, Finished, Available)

Cargar el dataset de Yelp

In [80]:
hotels_yelp = pd.read_csv("/lakehouse/default/Files/data/beta/Yelp/business.csv")
hotels_yelp = hotels_yelp.iloc[:,1:]
hotels_yelp.drop_duplicates(subset=['business_id'], inplace=True, ignore_index=True)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 82, Finished, Available)

In [46]:
hotels_yelp.info()

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 48, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1592 entries, 0 to 1591
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1592 non-null   int64  
 1   business_id   1592 non-null   object 
 2   name          1592 non-null   object 
 3   address       1592 non-null   object 
 4   city          1592 non-null   object 
 5   postal_code   1592 non-null   int64  
 6   latitude      1592 non-null   float64
 7   longitude     1592 non-null   float64
 8   stars         1592 non-null   float64
 9   review_count  1592 non-null   int64  
 10  is_open       1592 non-null   int64  
 11  attributes    1486 non-null   object 
 12  categories    1592 non-null   object 
 13  hours         1233 non-null   object 
 14  state         1592 non-null   object 
dtypes: float64(3), int64(4), object(8)
memory usage: 186.7+ KB


Cargar el dataset de Hotel Beds

In [6]:
hotels_hb = pd.read_csv("/lakehouse/default/Files/data/beta/Hotelbeds/hotels_hb_dataset.csv")

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 8, Finished, Available)

Crear objetos GeoDataFrame a partir de las coordenadasde de los dataframes (latitud y longitud)

In [47]:
geometry_gm = [Point(xy) for xy in zip(hotels_gm['longitude'], hotels_gm['latitude'])]
geometry_yelp = [Point(xy) for xy in zip(hotels_yelp['longitude'], hotels_yelp['latitude'])]
geometry_hb = [Point(xy) for xy in zip(hotels_hb['longitude'], hotels_hb['latitude'])]

gdf_gm = gpd.GeoDataFrame(hotels_gm, geometry=geometry_gm)
gdf_yelp = gpd.GeoDataFrame(hotels_yelp, geometry=geometry_yelp)
gdf_hb = gpd.GeoDataFrame(hotels_hb[['hotel_id', 'name', 'longitude', 'latitude']], geometry=geometry_hb)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 49, Finished, Available)

Realizar la unión espacial de los dataframes basada en la proximidad de coordenadas

>**`Unión de Google Maps con Hotel Beds`**

In [10]:
joined_gm = gpd.sjoin_nearest(gdf_gm, gdf_hb, how="inner", max_distance=0.001)
joined_gm.drop_duplicates(subset=["hotel_id"],inplace=True)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 12, Finished, Available)

Crear 2 columnas que resten la latitud y longitud de los 2 dataframes unidos/cruzados anteriormente para conocer cuál(es) registro(s) posee(n) la máxima diferencia de distancia o ubicación y corroborar que corresponda al mismo hotel. 

In [11]:
joined_gm['subs_latitude'] = joined_gm['latitude_left'] - joined_gm['latitude_right']
joined_gm['subs_longitude'] = joined_gm['longitude_left'] - joined_gm['longitude_right']

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 13, Finished, Available)

Obtener información (el máximo específicamente) de las columnas creadas anteriormente

In [12]:
joined_gm.loc[:, ['subs_latitude', 'subs_longitude']].describe()

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 14, Finished, Available)

,subs_latitude,subs_longitude
count,5.100000e+02,5.100000e+02
mean,1.175325e-05,3.885647e-06
std,3.187687e-04,3.353880e-04
min,-9.297154e-04,-9.133000e-04
25%,-1.379750e-04,-1.681750e-04
50%,1.500000e-08,1.000000e-07
75%,1.466084e-04,1.760284e-04
max,9.351000e-04,9.471000e-04


Se observa que la máxima diferencia entre latitudes es de 0.000935 y en longitudes 0.000947. También es importante acotar que de 1422 registros del dataset de Google Maps se cruzaron 464 con el dataset de Hotel Beds.

La máxima diferencia de latitud y longitud se da en los siguientes hoteles: 

In [13]:
joined_gm[joined_gm['subs_latitude'] == joined_gm['subs_latitude'].max()]

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 15, Finished, Available)

,name_left,address,gmap_id,description,latitude_left,longitude_left,category,rating_gm,review_count_gm,MISC,...,category_hotel,state_gm,geometry,index_right,hotel_id,name_right,longitude_right,latitude_right,subs_latitude,subs_longitude
1266,The Ridge Motel is remodeled and Rebranded wit...,The Ridge Motel is remodeled and Rebranded wit...,0x52d7860566778957:0xfda09315c43a9acf,Kids under 16 stay free with parents in a basi...,46.849905,-100.914458,['Hotel'],3.3,68,NaN,...,Hotel,North_Dakota,POINT (-100.91446 46.84991),27156,490416,Baymont by Wyndham Mandan Bismarck Area,-100.91469,46.84897,0.000935,0.000232


In [14]:
joined_gm[joined_gm['subs_longitude'] == joined_gm['subs_longitude'].max()]

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 16, Finished, Available)

,name_left,address,gmap_id,description,latitude_left,longitude_left,category,rating_gm,review_count_gm,MISC,...,category_hotel,state_gm,geometry,index_right,hotel_id,name_right,longitude_right,latitude_right,subs_latitude,subs_longitude
1319,Best Western Plus Executive Residency Columbus...,Best Western Plus Executive Residency Columbus...,0x8838f3434bb53ecf:0x3cd9c935046142c0,NaN,40.114057,-83.004518,['Hotel'],4.1,58,NaN,...,Hotel,Ohio,POINT (-83.00452 40.11406),14682,251914,Mainstay Suites Columbus Worthington,-83.005465,40.114143,-0.000086,0.000947


Al consultar via web la longitud y latitud de cada dataset por separado se corrobora que corresponden al mismo hotel.

>**`Unión de Yelp con Hotel Beds`**

Se hace el mismo procedimiento realizado en la unión de _Google Maps_ con _Hotel Beds_

In [48]:
joined_yelp = gpd.sjoin_nearest(gdf_yelp, gdf_hb, how="inner", max_distance=0.001)
joined_yelp.drop_duplicates(subset=["hotel_id"],inplace=True)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 50, Finished, Available)

In [49]:
joined_yelp['resta_latitude'] = joined_yelp['latitude_left'] - joined_yelp['latitude_right']
joined_yelp['resta_longitude'] = joined_yelp['longitude_left'] - joined_yelp['longitude_right']

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 51, Finished, Available)

In [50]:
joined_yelp.loc[:, ['resta_latitude', 'resta_longitude']].describe()

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 52, Finished, Available)

,resta_latitude,resta_longitude
count,9.750000e+02,975.000000
mean,3.011836e-07,-0.000012
std,2.765087e-04,0.000327
min,-9.339000e-04,-0.000996
25%,-1.238625e-04,-0.000169
50%,0.000000e+00,-0.000007
75%,1.383639e-04,0.000142
max,9.655298e-04,0.000966


Se observa que la máxima diferencia entre latitudes es de 0.000966 y en longitudes 0.000966. También es importante acotar que de 1592 registros del dataset de Yelp se cruzaron 1131 con el dataset de Hotel Beds.

La máxima diferencia de latitud y longitud se da en los siguientes hoteles:

In [51]:
joined_yelp[joined_yelp['resta_latitude'] == joined_yelp['resta_latitude'].max()]

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 53, Finished, Available)

,Unnamed: 0,business_id,name_left,address,city,postal_code,latitude_left,longitude_left,stars,review_count,...,hours,state,geometry,index_right,hotel_id,name_right,longitude_right,latitude_right,resta_latitude,resta_longitude
735,735,edBkcpIZlX6tkXUc0A2e0Q,Courtyard Indianapolis at the Capitol,320 North Senate Avenue,Indianapolis,46204,39.77301,-86.164672,3.5,38,...,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",Indiana,POINT (-86.16467 39.77301),8615,252240,Residence Inn Indianapolis Downtown on the Canal,-86.164731,39.772044,0.000966,0.000059


In [52]:
joined_yelp[joined_yelp['resta_longitude'] == joined_yelp['resta_longitude'].max()]

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 54, Finished, Available)

,Unnamed: 0,business_id,name_left,address,city,postal_code,latitude_left,longitude_left,stars,review_count,...,hours,state,geometry,index_right,hotel_id,name_right,longitude_right,latitude_right,resta_latitude,resta_longitude
442,442,be6q2TFhFqUQtR13n5cxEQ,Hampton Inn & Suites Deptford,1253 Hurffville Rd,Deptford,8096,39.828194,-75.087963,2.5,5,...,NaN,Nueva Jersey,POINT (-75.08796 39.82819),39785,878780,"Hampton Inn & Suites Deptford, NJ",-75.088928,39.827999,0.000195,0.000966


Al consultar via web la longitud y latitud de cada dataset por separado se corrobora que corresponden al mismo hotel.

Por otra parte, se crea una columna llamada _hotel-id_ a la cual se le asigna el mismo valor de la columna _code_ de _Hotels Beds_ y así tener una clave primaria para normalizar los datos

In [ ]:
joined_gm = joined_gm.loc[:, ['hotel_id', 'gmap_id', 'name_left', 'address', 'description', 'latitude_left', 
                               'longitude_left', 'state_gm', 'category', 'rating_gm', 'review_count_gm', 'MISC',
                               'relative_results', 'url']].rename(columns={'name_left':'name', 'latitude_left':'latitude', 'longitude_left':'longitude'})


In [54]:
joined_yelp = joined_yelp.loc[:, ['hotel_id', 'business_id', 'name_left', 'address', 'city', 'postal_code', 'latitude_left',
                                  'longitude_left', 'stars', 'review_count', 'is_open', 'attributes',
                                  'categories', 'hours', 'state']].rename(columns={'name_left':'name', 'latitude_left':'latitude',
                                                                                   'longitude_left':'longitude'})


StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 56, Finished, Available)

## **Proceso de concatenado de Google Maps**

In [97]:
# Generamos una copia de hoteles de google maps para proceder a gestionar la tabla antes 
# de ser normalizada junto a hotelbeds en otro Notebook
hotels_copy_gm = hotels_gm.copy()
hotels_copy_gm['hotel_id'] = ''

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 99, Finished, Available)

In [98]:
# Seleccionamos el mismo orden de los registros ya relacionados de google maps con hotelbeds por cordenadas
hotels_copy_gm = hotels_copy_gm.loc[:, ['hotel_id', 'gmap_id', 'name', 'address', 'description', 'latitude', 
                               'longitude', 'state_gm', 'category', 'rating_gm', 'review_count_gm', 'MISC',
                               'relative_results', 'url']]

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 100, Finished, Available)

In [99]:
# Generamos una copia al dataset relacionado por cordenadas de google maps con hotelbeds
joined_copy_gm = joined_gm.copy()

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 101, Finished, Available)

#### Realizamos un filtrado eliminando del dataframe `hotels_copy_gm` (data completa) los registros de `joined_copy_gm` (data relacionada con hotel beds) para que no queden duplicados ya que también se deben tener en cuenta y luego asignarles un `hotel_id`

In [102]:
# Elimina las filas coincidentes de hotels_copy_gm
hotels_copy_gm = hotels_copy_gm.drop(joined_copy_gm.index)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 104, Finished, Available)

In [103]:
# Asignamos un ID a los que están vacíos y no se encontraron en la data de hotelbeds
hotels_copy_gm['hotel_id'] = range(1010161951, 1010161951 + len(hotels_copy_gm))

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 105, Finished, Available)

In [104]:
# Concatenamos los dos dataframes de google maps ya terminados
hotels_all_gm = pd.concat([hotels_copy_gm, joined_copy_gm], ignore_index=True)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 106, Finished, Available)

In [106]:
# Guardamos a un dataset de google maps previo a normalizarlo con la tabla de hoteles principal
hotels_all_gm.to_csv("/lakehouse/default/Files/data/beta/Google/hotels_gm_hb.csv", index=False)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 108, Finished, Available)

In [105]:
hotels_all_gm.info()

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 107, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556 entries, 0 to 1555
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hotel_id          1556 non-null   int64  
 1   gmap_id           1556 non-null   object 
 2   name              1556 non-null   object 
 3   address           1555 non-null   object 
 4   description       770 non-null    object 
 5   latitude          1556 non-null   float64
 6   longitude         1556 non-null   float64
 7   state_gm          1556 non-null   object 
 8   category          1556 non-null   object 
 9   rating_gm         1556 non-null   float64
 10  review_count_gm   1556 non-null   int64  
 11  MISC              106 non-null    object 
 12  relative_results  1507 non-null   object 
 13  url               1556 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 170.3+ KB


## **Proceso de concatenado de Yelp**

In [84]:
# Generamos una copia de hoteles de Yelp para proceder a gestionar la tabla antes 
# de ser normalizada junto a hotelbeds en otro Notebook
hotels_copy_yelp = hotels_yelp.copy()
hotels_copy_yelp['hotel_id'] = ''

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 86, Finished, Available)

In [85]:
# Seleccionamos el mismo orden de los registros ya relacionados de yelp con hotelbeds por cordenadas
hotels_copy_yelp = hotels_copy_yelp.loc[:, ['hotel_id', 'business_id', 'name', 'address', 'city', 'postal_code', 'latitude',
                                  'longitude', 'stars', 'review_count', 'is_open', 'attributes',
                                  'categories', 'hours', 'state']]


StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 87, Finished, Available)

In [86]:
# Generamos una copia al dataset relacionado por cordenadas de yelp con hotelbeds
joined_copy_yelp = joined_yelp.copy()

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 88, Finished, Available)

#### Realizamos un filtrado eliminando del dataframe `hotels_copy_yelp` (data completa) los registros de `joined_copy_yelp` (data relacionada con hotel beds) para que no queden duplicados ya que también se deben tener en cuenta y luego asignarles un `hotel_id`

In [89]:
# Elimina las filas coincidentes de hotels_copy_yelp
hotels_copy_yelp = hotels_copy_yelp.drop(joined_copy_yelp.index)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 91, Finished, Available)

In [92]:
# Asignamos un ID a los que están vacíos y no se encontraron en la data de hotelbeds
hotels_copy_yelp['hotel_id'] = range(1034670504, 1034670504 + len(hotels_copy_yelp))

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 94, Finished, Available)

In [93]:
# Concatenamos los dos dataframes de yelp ya terminados
hotels_all_yelp = pd.concat([hotels_copy_yelp, joined_copy_yelp], ignore_index=True)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 95, Finished, Available)

In [95]:
# Guardamos a un dataset de yelp previo a normalizarlo con la tabla de hoteles principal
hotels_all_yelp.to_csv("/lakehouse/default/Files/data/beta/Yelp/hotels_yelp_hb.csv", index=False)

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 97, Finished, Available)

In [94]:
hotels_all_yelp.info()

StatementMeta(, aa7ac24c-743e-4a87-8fae-b2a0620d719c, 96, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hotel_id      1593 non-null   int64  
 1   business_id   1593 non-null   object 
 2   name          1593 non-null   object 
 3   address       1593 non-null   object 
 4   city          1593 non-null   object 
 5   postal_code   1593 non-null   int64  
 6   latitude      1593 non-null   float64
 7   longitude     1593 non-null   float64
 8   stars         1593 non-null   float64
 9   review_count  1593 non-null   int64  
 10  is_open       1593 non-null   int64  
 11  attributes    1487 non-null   object 
 12  categories    1593 non-null   object 
 13  hours         1234 non-null   object 
 14  state         1593 non-null   object 
dtypes: float64(3), int64(4), object(8)
memory usage: 186.8+ KB
